In [ ]:
!pip install pulp

     |████████████████████████████████| 40.6MB 109kB/s 


In [ ]:
from pulp import *
import pandas as pd

  #dostępni pracownicy
pracownicy = ["Ania","Stefan","Hektor","Olaf","Lidia","Piotr"]
  #płaca za godzinę
cena_h = [100,50,60,40,110,70]
  #dni pracujące
dni = ["Pon","Wt","Sr","Czw","Pia"]
  #ilośc pracowników w danym dniu
ilosc_pracownikow_na_dzien = [2,1,1,1,3]
  #sumowanie elementów
sum_tab = [] 
  #lista informująca o dostępności pracownika każdego dnia
prac_dzien = []
  #lista informująca kiedy dany pracownik nie może się stawić w pracy
nie_moze_w = [[],[1,2,3,4],[3,4],[],[0,1,2],[3,4]]

  #tworzenie zmiennej zawierającej dane problemu
prob = LpProblem("Sklep", LpMinimize)

  #wypełnianie tablicy nformującej o dostępności pracownika każdego dnia
for pracownik in pracownicy:
    pracownik = [LpVariable(pracownik+"{}".format(dzien), cat = "Binary") for dzien in dni]
    prac_dzien.append(pracownik)

  #ograniczenia
    #- koszty zatrudnienia pracowników
prob += sum((8*(cena*(sum(x))) for x,cena in zip(prac_dzien,cena_h)))

    #- dla każdego pracownika kiedy może pracować
for numer,pracownik in enumerate(prac_dzien):
    for nie_moze in nie_moze_w[numer]:
        sum_tab.append(pracownik[nie_moze])
    prob += sum(sum_tab) == 0 
    sum_tab.clear()

    #- dla każdego pracownika kiedy nie może pracować
for dzien in range(len(dni)):
    for pracownik in range(len(pracownicy)):
        sum_tab.append(prac_dzien[pracownik][dzien])
    prob += sum(sum_tab) == ilosc_pracownikow_na_dzien[dzien]
    sum_tab.clear()

    #- maksymalna iloścp racowników na dzień
for pracownik in prac_dzien:
    prob += sum(pracownik) <= 3

  #zapisanie danych do pliku .lp
prob.writeLP("Scheduling_Problem")

  #rozwiązanie problemu w pulp'ie
prob.solve()

  #wyświetlenie statusu rozwiązania
print ("Status:", LpStatus[prob.status])

  #wyświetlenie zmiennych z ich optymalną wartością
for v in prob.variables():
    if(v.varValue == 1):
        print (v.name, "=", v.varValue)

  #wyświetlenie minimalnych kosztów zatrudnienia pracowników
print("Min. koszt = ", value(prob.objective))

Status: Optimal
AniaPia = 1.0
HektorPon = 1.0
HektorSr = 1.0
LidiaPia = 1.0
OlafCzw = 1.0
OlafPia = 1.0
OlafWt = 1.0
StefanPon = 1.0
Min. koszt =  4000.0
